In [1]:
import pandas as pd
import numpy as np
import paho.mqtt.subscribe as subscribe
import paho.mqtt.publish as publish
import json
import joblib
from tensorflow.keras.models import load_model

In [2]:
#Sequence length
seq_len = 10

filename = 'N-CMAPSS_DS03-012.h5'

dataset = filename[-11:-3]

X_columns = 14
Xw_columns = 18

topics = ["diagnostics", "prognostics"]

new_unit = 0

path = './deep_learning_models/'

In [3]:
def scaling(data):

    scaler = joblib.load(path + f'scaler_{dataset}.gz')
    
    Xw_test_scaled = pd.DataFrame(scaler.transform(data.iloc[:, -Xw_columns:]), columns=data.iloc[:, -Xw_columns:].columns)
    
    return Xw_test_scaled

In [4]:
def state_shaping(Xw_test_scaled, state):

    mod = len(Xw_test_scaled) % seq_len
    ran = len(Xw_test_scaled) // seq_len
    

    X_cycle = Xw_test_scaled.iloc[:, -X_columns:].to_numpy()

    del Xw_test_scaled
    
    y_state = np.full((ran, 1), state)
     
    X_state = np.empty((0,seq_len,X_columns))
            
    for idx in range(0, ran):  
            X_state = np.vstack((X_state, X_cycle[(mod + idx*seq_len): mod+seq_len + idx*seq_len].reshape(1, seq_len, X_columns)))
    
    return X_state, y_state

In [5]:
def rul_shaping(Xw_test_scaled, rul):

    global c_idx
    ruls_inf = [30, 15, 0]
    classes = [2, 1, 0]

    mod = len(Xw_test_scaled) % seq_len
    ran = len(Xw_test_scaled) // seq_len
    
    X_cycle = Xw_test_scaled.iloc[:, -Xw_columns:].to_numpy()

    del Xw_test_scaled
     
    X_rul = np.empty((0,seq_len,Xw_columns))

    if c_idx == -1:
        for c_idx, value in enumerate(ruls_inf):             
            if value <= rul:
                break
            
    y_rul = np.full((ran, 1), classes[c_idx])

    if (rul == ruls_inf[c_idx]):
        c_idx += 1
            
    for idx in range(0, ran):  
            X_rul = np.vstack((X_rul, X_cycle[(mod + idx*seq_len): mod+seq_len + idx*seq_len].reshape(1, seq_len, Xw_columns)))
    
    return X_rul, y_rul

In [6]:
def evaluating(X_test, y_test, topic):
    
    if topic == 'diagnostics':
        model = load_model(path + f'state_{dataset}_model_{seq_len}.h5')
    
    if topic == 'prognostics':
        model = load_model(path + f'rul_{dataset}_model_{seq_len}.h5')

    # Classes
    pred = model.predict(X_test)
    y_pred = np.argmax(pred, axis=1)
    state = np.bincount(y_pred).argmax()
    
    tot = len(X_test)
    correct = 0

    for i in range(tot):
        if y_test[i] == y_pred[i]:
            correct = correct + 1

    accuracy = round((correct / tot * 100), 2)
    
    return state, accuracy

In [7]:
def wait_for_data(client, userdata, msg):
    
    global data, unit, new_unit, cycle, c_idx

    if msg.topic == 'diagnostics':
        
        data = pd.read_json(msg.payload.decode())

        unit = data["Unit"].unique()[0]
        cycle = data["Cycle"].unique()[0]
        
        print(f'Unit: {unit} and cycle: {cycle}')
    
        #Send to dashboard
        info_msg = json.dumps({'unit': int(unit), 'cycle': int(cycle)})
        publish.single("unit", payload=info_msg, qos=1, hostname="localhost")
        
        Xw_test_scaled = scaling(data)   
        X_state, y_state = state_shaping(Xw_test_scaled, data['State'].iloc[0])
        state, accuracy = evaluating(X_state, y_state, msg.topic)
        
        model_msg = json.dumps({'state': int(state), 'accuracy': accuracy})
        publish.single("state", payload=model_msg, qos=1, hostname="localhost")
        print(f'State: {state} and accuracy: {accuracy}%\n')
            
    if msg.topic == 'prognostics':
        
        print(f'Unit: {unit} and cycle: {cycle}')

        if new_unit != unit:
            c_idx = -1
            new_unit = unit

        Xw_test_scaled = scaling(data)

        X_rul, y_rul = rul_shaping(Xw_test_scaled, data['RUL'].iloc[0])

        rul, accuracy = evaluating(X_rul, y_rul, msg.topic)

        model_msg = json.dumps({'rul': int(rul), 'accuracy': accuracy})
        publish.single("rul", payload=model_msg, qos=1, hostname="localhost")
        print(f'RUL class: {rul} and accuracy: {accuracy}%\n')


In [ ]:
try:
    subscribe.callback(wait_for_data, topics, qos=1, hostname="localhost")
except ConnectionRefusedError:
    print('Broker may not be available')

Unit: 13 and cycle: 1
48/48 [==============================] - 2s 5ms/step
State: 1 and accuracy: 97.35%

Unit: 13 and cycle: 2
47/47 [==============================] - 2s 5ms/step
State: 1 and accuracy: 99.39%

Unit: 13 and cycle: 3
43/43 [==============================] - 2s 5ms/step
State: 1 and accuracy: 92.6%

Unit: 13 and cycle: 4
45/45 [==============================] - 2s 5ms/step
State: 1 and accuracy: 97.43%

Unit: 13 and cycle: 5
33/33 [==============================] - 2s 5ms/step
State: 1 and accuracy: 97.57%

Unit: 13 and cycle: 6
39/39 [==============================] - 2s 5ms/step
State: 1 and accuracy: 96.3%

Unit: 13 and cycle: 7
55/55 [==============================] - 3s 5ms/step
State: 1 and accuracy: 94.6%

Unit: 13 and cycle: 8
28/28 [==============================] - 2s 5ms/step
State: 1 and accuracy: 99.89%

Unit: 13 and cycle: 9
48/48 [==============================] - 2s 5ms/step
State: 1 and accuracy: 95.27%

Unit: 13 and cycle: 10
55/55 [===================